In [16]:
from music21 import chord, harmony, interval
import json

In [17]:
def transpose(ch, key):
  # transpose the given chord (wrt given key) to the C major scale
  return ch.transpose(interval.Interval(-key.normalOrder[0]))

def tomusic21(ch):
  # flatten human-written chords into music21 chord objects
  return [chord.Chord(harmony.ChordSymbol(c)) for c in ch]

def flatten(ch):
  # flatten music21 chords into normal order vectors
  return [c.normalOrder for c in ch]

def transpose_chs(chs, key):
  # make into actual chords, and transpose each wrt the song's key
  key = chord.Chord(harmony.ChordSymbol(key))
  return [transpose(c, key) for c in chs]

In [18]:
print(flatten(tomusic21(['C', 'F', 'G', 'C'])))
print((transpose_chs(tomusic21(['F', 'B-', 'C', 'F']), 'F')))

[[0, 4, 7], [5, 9, 0], [7, 11, 2], [0, 4, 7]]
[<music21.chord.Chord C3 E3 G3>, <music21.chord.Chord F2 A2 C3>, <music21.chord.Chord G2 B2 D3>, <music21.chord.Chord C3 E3 G3>]


In [3]:
def sanitize_chord(ch):
    reps = {
        '\\': '',
        'b': '-',
        '-5': 'b5',
        '-9': 'b9',
        '(maj7)': 'M7',
        '(+5)': 'add5',
        '+5': 'add5',
        '/9': 'add9',
        'maj9': 'add9',
        'sus2': 'add2',
        'add4add9':'add4',
        'Maj7':'M7',
        '/G#':'b9',
        '(2)':'add2',
    }
    for k, v in reps.items():
        ch = ch.replace(k, v)
    return ch

def preprocess(fname,out_file):
    import sys
    print('Preprocessing dataset...')
    print('--------------------------------')
    
    count = 0
    songs = ''
    with open(fname, 'r') as f:
        j = json.loads(f.read())
        for song in j[count:]:
            chs = [sanitize_chord(c) for c in song['chords']]
            chs = tomusic21(chs)
            if(song['tonality_name'] == ""):
                continue
            chs = transpose_chs(chs,sanitize_chord(song['tonality_name']))
            chs = flatten(chs) #List of chords in a song 
            procs = [' '.join([str(n) for n in ch]) for ch in chs]
            procs = ' - '.join(procs)
            songs = songs + procs + ' . '
            if count % 50 == 0:
                sys.stdout.write('{}... '.format(count))
            count += 1
    g = open(out_file,'w')
    g.write(songs)
    g.close()
    print('--------------------------------')
    
def better_preprocess(fname, outfname):
    import sys
    from functools import reduce
    
    print('Better preprocessing dataset...')
    print('--------------------------------')
    
    count = 0
    final = []
    
    with open(fname, 'r') as f:
        j = json.loads(f.read())
        
        for song in j[count:]:
            chs = [sanitize_chord(c) for c in song['chords']]
            chs = tomusic21(chs)
            key = sanitize_chord(song['tonality_name'])
            if key == '':
                continue
                
            chs = transpose_chs(chs, key)
            chs = flatten(chs)
            # separate chords with '12' integer
            chs = reduce(lambda a, b: a+[12]+b, chs)
            final.append(chs)
            if count % 50 == 0:
                sys.stdout.write('{}... '.format(count))
            count += 1
    
    print('--------------------------------')
    
    # separate songs with '13' integer
    final = reduce(lambda a, b: a+[13]+b, final)
    
    with open(outfname, 'w') as f:
        json.dump(final, f)

In [132]:
def count_none(fn):
    count = 0 
    with open(fn,'r') as f:
        j = json.loads(f.read())
        for song in j:
            if(song['tonality_name'] == ''):
                count += 1
    return count

#count_none('json_songs.json')

In [148]:
preprocess('json_songs.json', 'test')
better_preprocess('json_songs.json', 'clean_dataset.json')

Preprocessing dataset...
--------------------------------
0... 50... 100... 150... 200... 250... 300... 350... 400... 450... 500... 550... 600... 650... --------------------------------
Better preprocessing dataset...
--------------------------------
0... 50... 100... 150... 200... 250... 300... 350... 400... 450... 500... 550... 600... 650... --------------------------------


In [4]:
tomusic21(['Am'])[0]

<music21.chord.Chord A2 C3 E3>

In [5]:
z = chord.Chord(['D3', 'F#3','G3','A3','E3'])
harmony.chordSymbolFigureFromChord(z,False)

'DaddG,E'

In [ ]:
chord.Chord(harmony.ChordSymbol('A-'))

In [23]:
def find_song(fname, num):
    with open(fname,'r') as f:
        j = json.loads(f.read())
        raw = j[num]
        chs = [sanitize_chord(c) for c in raw['chords']]
        chs = tomusic21(chs)
#         print(chs)
        return chs, raw['tonality_name']
        
find_song('json_songs.json',201)

([<music21.chord.Chord E3 G#3 B3>,
  <music21.chord.Chord A2 C#3 E3>,
  <music21.chord.Chord B2 D#3 F#3>,
  <music21.chord.Chord G#3 B3 D#4>,
  <music21.chord.Chord C#3 E3 G#3>,
  <music21.chord.Chord A2 C#3 E3>,
  <music21.chord.Chord B2 D#3 F#3>,
  <music21.chord.Chord G#3 B3 D#4>,
  <music21.chord.Chord C#3 E3 G#3>,
  <music21.chord.Chord A2 C#3 E3>,
  <music21.chord.Chord B2 D#3 F#3>,
  <music21.chord.Chord G#3 B3 D#4>,
  <music21.chord.Chord C#3 E3 G#3>,
  <music21.chord.Chord A2 C#3 E3>,
  <music21.chord.Chord B2 D#3 F#3>,
  <music21.chord.Chord G#3 B3 D#4>,
  <music21.chord.Chord C#3 E3 G#3>,
  <music21.chord.Chord G3 B3 D4>,
  <music21.chord.Chord C3 E3 G3>,
  <music21.chord.Chord D3 F#3 A3>,
  <music21.chord.Chord B2 D3 F#3>,
  <music21.chord.Chord E3 G3 B3>,
  <music21.chord.Chord C3 E3 G3>,
  <music21.chord.Chord D3 F#3 A3>,
  <music21.chord.Chord B2 D3 F#3>,
  <music21.chord.Chord E3 G3 B3>,
  <music21.chord.Chord A2 C#3 E3>,
  <music21.chord.Chord B2 D#3 F#3>,
  <music21.c

In [71]:
c = chord.Chord(['C', 'E-', 'G', 'B-'])

In [6]:
from music21 import midi, volume, stream

In [14]:
# paste the output of the NN in here, and it will generate a midi file with the chord progression

# old method
# -------------------
# outp = '10 2 5 - 0 4 7 - 9 0 4 - 2 6 9 - 5 8 0 - 0 4 7 - 0 4 7 - 7 11 2 - 0 4 7 - 0 4 7 - 2 5 9 - 7 11 2 - 0 4 7 - 0 4 7 - 5 9 0 - 7 11 2 - 10 2 5 - 7 11 2 - 2 6 9 - 5 9 0 - 0 4 7 - 5 9 0 - 0 4 7 - 2 5 9 - 7 11 2 - 0 4 7 - 5 9 0 - 0 4 7 - 2 5 9 - 7 11 2 - 0 4 7 - 5 '
# chords = outp.split('-')
# chords = [[int(c) for c in ch.split()] for ch in chords]
# -------------------

def timenow():
    import time
    return time.strftime("%Y%m%d-%H%M%S")

chords = [[7, 11, 2, 13, 0, 4, 7],
 [2, 6, 9],
 [11, 3, 6],
 [4, 8, 8, 11],
 [6, 8, 1, 3],
 [11, 2, 6],
 [10, 1, 6],
 [1, 5, 8],
 [11, 2, 5, 8],
 [8, 11, 3],
 [6, 10, 1],
 [1, 6, 8],
 [1, 5, 8],
 [1, 5, 8],
 [1, 3, 6],
 [10, 1, 5],
 [10, 1, 3]]
chords = [ chord.Chord(ch) for ch in chords ]

off = 0.0
for ch in chords:
    ch.volume = volume.Volume(velocity=90)
    ch.volume.velocityIsRelative = False
    ch.offset = off
    off += 1.2
    
s = stream.Stream(chords)
print(s)
mf = midi.translate.streamToMidiFile(s)
mf.open('{}_0_9.midi'.format(timenow()), 'wb')
mf.write()
mf.close()

<music21.stream.Stream 0x1ad084bf630>
